In [1]:
#!pip install fastapi
#!pip install pydantic

In [3]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import numpy as np
import pickle

api = FastAPI()

# Load the trained model
with open("Linear_regression_housing_case_study.pkl", "rb") as model_file:
    model = pickle.load(model_file)

# Define the expected input structure
class ModelInput(BaseModel):
    area: float
    bedrooms: int
    bathrooms: int
    stories: int
    mainroad: int         # 1 if yes, 0 if no
    guestroom: int        # 1 if yes, 0 if no
    hotwaterheating: int  # 1 if yes, 0 if no
    airconditioning: int  # 1 if yes, 0 if no
    parking: int
    prefarea: int         # 1 if yes, 0 if no

@api.post("/predict")
def predict(input_data: ModelInput):
    try:
        # Convert input to a NumPy array and reshape for prediction
        features = np.array([[
            input_data.area,
            input_data.bedrooms,
            input_data.bathrooms,
            input_data.stories,
            input_data.mainroad,
            input_data.guestroom,
            input_data.hotwaterheating,
            input_data.airconditioning,
            input_data.parking,
            input_data.prefarea
        ]])

        # Make prediction
        prediction = model.predict(features)
        return {"predicted_price": int(prediction[0])}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction failed: {e}")


SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (1740357750.py, line 22)

## Visualization Dashboard

In [5]:
!pip install streamlit
!pip install uvicorn

In [ ]:
import streamlit as st
import requests
import pandas as pd

# Define the FastAPI endpoint
API_URL = "http://127.0.0.1:8000/predict"  # Replace with actual URL if deployed

# App title
st.title("🏠 Housing Price Prediction Dashboard")

st.markdown("Enter housing features below to get a predicted price:")

# Input fields for all features
area = st.number_input("Area (sq ft)", value=3000)
bedrooms = st.number_input("Number of Bedrooms", min_value=0, value=3)
bathrooms = st.number_input("Number of Bathrooms", min_value=0, value=2)
stories = st.number_input("Number of Stories", min_value=1, value=2)
mainroad = st.selectbox("Main Road Access", ["Yes", "No"])
guestroom = st.selectbox("Guest Room", ["Yes", "No"])
hotwaterheating = st.selectbox("Hot Water Heating", ["Yes", "No"])
airconditioning = st.selectbox("Air Conditioning", ["Yes", "No"])
parking = st.number_input("Number of Parking Spots", min_value=0, value=1)
prefarea = st.selectbox("Preferred Area", ["Yes", "No"])

# Convert categorical inputs to binary
mainroad_val = 1 if mainroad == "Yes" else 0
guestroom_val = 1 if guestroom == "Yes" else 0
hotwater_val = 1 if hotwaterheating == "Yes" else 0
aircon_val = 1 if airconditioning == "Yes" else 0
prefarea_val = 1 if prefarea == "Yes" else 0

# Input data dictionary
input_data = {
    "area": area,
    "bedrooms": bedrooms,
    "bathrooms": bathrooms,
    "stories": stories,
    "mainroad": mainroad_val,
    "guestroom": guestroom_val,
    "hotwaterheating": hotwater_val,
    "airconditioning": aircon_val,
    "parking": parking,
    "prefarea": prefarea_val
}

# Show input summary as a table
st.subheader("🔎 Feature Summary")
st.dataframe(pd.DataFrame([input_data]))

# Button to make prediction
if st.button("Predict Price 💰"):
    try:
        response = requests.post(API_URL, json=input_data)
        if response.status_code == 200:
            result = response.json()
            st.success(f"🏷️ Predicted House Price: ₹{result['predicted_price']:,}")
        else:
            st.error(f"API Error: {response.status_code} - {response.text}")
    except Exception as e:
        st.error(f"Connection error: {e}")
